In [3]:
import os

def calculate_game_state(move):
    victories = {"Rock": "Scissors", "Paper": "Rock", "Scissors": "Paper"}
    moves = ("Rock", "Paper", "Scissors")
    computer_move = random.choice(moves)

    if computer_move == move:
        return 0, computer_move
    if victories[move] == computer_move:
        return 1, computer_move
    return -1, computer_move

def get_finger_status(hands_module, hand_landmarks, finger_name):
    finger_id_map = {'INDEX': 8, 'MIDDLE': 12, 'RING': 16, 'PINKY': 20}
    finger_tip_y = hand_landmarks.landmark[finger_id_map[finger_name]].y
    finger_mcp_y = hand_landmarks.landmark[finger_id_map[finger_name] - 2].y

    return finger_tip_y < finger_mcp_y


def get_thumb_status(hands_module, hand_landmarks):
    thumb_tip_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP].x
    thumb_mcp_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP - 2].x
    thumb_ip_x = hand_landmarks.landmark[hands_module.HandLandmark.THUMB_TIP - 1].x

    return thumb_tip_x > thumb_ip_x > thumb_mcp_x

def play_rps_game():
    drawing_module = mediapipe.solutions.drawing_utils
    hands_module = mediapipe.solutions.hands

    capture = cv2.VideoCapture(0)

    start_time = 0.0
    timer_started = False
    time_left = 2
    hold_for_play = False
    draw_timer = 0.0
    game_over_text = ""
    computer_played = ""
    player_wins = 0
    computer_wins = 0
    font = cv2.FONT_HERSHEY_COMPLEX   

    image1 = cv2.imread("images/imagen_inicio.jpg")
    image2 = cv2.imread("images/seleccion.jpg")
    image3 = cv2.imread("images/ganar.jpg")
    image4 = cv2.imread("images/perder.jpg")
    image5 = cv2.imread("images/empate.jpg")
    exit = False
    # Image to concat
    imAux = image1

    with hands_module.Hands(static_image_mode=False, min_detection_confidence=0.7,
                            min_tracking_confidence=0.4, max_num_hands=2) as hands:
        
        hand_detection_active = True
        game_started = False
        exit = False


        while True:

            if timer_started:
                now_time = time.time()
                time_elapsed = now_time - start_time
                if time_elapsed >= 1:
                    time_left -= 1
                    start_time = now_time
                    if time_left <= 0:
                        hold_for_play = True
                        timer_started = False
                        imAux = image2


            ret, frame = capture.read()
            frame = cv2.flip(frame, 1)
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            cv2.putText(frame,
                f"Wins: {player_wins} | Losses: {computer_wins}",
                (460, 20),
                font, 0.5,
                (0, 204, 102),
                2,
                cv2.LINE_4)

            move = "UNKNOWN"
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    if not game_started:
                        thumb_status = get_thumb_status(hands_module, hand_landmarks)
                        if thumb_status:
                            game_started = True
                            start_time = time.time()
                            timer_started = True
                            time_left = 3

                    if hold_for_play or time.time() - draw_timer <= 2:
                        drawing_module.draw_landmarks(frame, hand_landmarks, hands_module.HAND_CONNECTIONS)

                    current_state = ""
                    thumb_status = get_thumb_status(hands_module, hand_landmarks)
                    current_state += "1" if thumb_status else "0"

                    index_status = get_finger_status(hands_module, hand_landmarks, 'INDEX')
                    current_state += "1" if index_status else "0"

                    middle_status = get_finger_status(hands_module, hand_landmarks, 'MIDDLE')
                    current_state += "1" if middle_status else "0"

                    ring_status = get_finger_status(hands_module, hand_landmarks, 'RING')
                    current_state += "1" if ring_status else "0"

                    pinky_status = get_finger_status(hands_module, hand_landmarks, 'PINKY')
                    current_state += "1" if pinky_status else "0"
                    if current_state == "00000":
                        move = "Rock"
                    elif current_state == "11111":
                        move = "Paper"
                    elif current_state == "01100":
                        move = "Scissors"
                    else:
                        move = "UNKNOWN"

                    if not game_started:
                        if current_state == "00111":
                            move = "Exit"
                            exit = True

                if hold_for_play and move != "UNKNOWN" and not exit:
                    hold_for_play = False
                    draw_timer = time.time()
                    won, cmp_move = calculate_game_state(move)
                    computer_played = "You: " + move + " | Computer: " + cmp_move
                    if won == 1:
                        game_started = False
                        player_wins += 1
                        imAux = image3
                       
                    elif won == -1:
                        game_started = False
                        computer_wins += 1
                        imAux = image4
                    else:
                        game_started = False
                        imAux = image5

            if not hold_for_play and not timer_started:
                cv2.putText(frame,
                            computer_played,
                            (10, 450),
                            font, 0.75,
                            (0, 204, 102),
                            2,
                            cv2.LINE_4)

            start_trigger = ""
            if timer_started:
                start_trigger = "Starting in " + str(time_left)

            cv2.putText(frame,
                        start_trigger,
                        (150, 200),
                        font, 1,
                        (0, 204, 102),
                        2,
                        cv2.LINE_4)
            
            resized_image = cv2.resize(frame, (800, 600))
            if not hand_detection_active:
                imAux = image2

            if imAux.shape[0] != resized_image.shape[0]:
                # Resize imAux to have the same number of rows as frame
                imAux = cv2.resize(imAux, (resized_image.shape[1], resized_image.shape[0]))

            # Concatenate images horizontally
            n_image = cv2.hconcat([imAux, resized_image])

            cv2.imshow('Rock Paper Scissors!', n_image)

            key = cv2.waitKey(1)

            
            if exit or key == (27 or ord('q')) or exit:
                break

    cv2.destroyAllWindows()
    capture.release()